# Notebook 1.

In [4]:
import pandas as pd
pd.set_option("display.float_format", "{:.2f}".format)

path = r"C:\Users\HP\Downloads\penguins_classification.csv"
penguins = pd.read_csv(path)
penguins.head()

,Culmen Length (mm),Culmen Depth (mm),Species
0,39.10,18.70,Adelie
1,39.50,17.40,Adelie
2,40.30,18.00,Adelie
3,36.70,19.30,Adelie
4,39.30,20.60,Adelie


In [5]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Culmen Length (mm)  342 non-null    float64
 1   Culmen Depth (mm)   342 non-null    float64
 2   Species             342 non-null    object 
dtypes: float64(2), object(1)
memory usage: 8.1+ KB
